## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qaanaaq,GL,77.4840,-69.3632,5.99,78,1,5.57,clear sky
1,1,Rio Grande,BR,-32.0350,-52.0986,66.00,88,1,7.61,clear sky
2,2,Upernavik,GL,72.7868,-56.1549,7.90,82,39,3.09,scattered clouds
3,3,Ketchikan,US,55.3422,-131.6461,42.80,56,1,3.44,clear sky
4,4,Mar Del Plata,AR,-38.0023,-57.5575,71.60,73,0,10.36,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Mar Del Plata,AR,-38.0023,-57.5575,71.60,73,0,10.36,clear sky
5,5,Atuona,PF,-9.8000,-139.0333,79.63,79,35,16.35,light rain
6,6,Gewane,ET,10.1664,40.6453,74.91,65,0,3.06,clear sky
16,16,Honiara,SB,-9.4333,159.9500,81.09,75,99,1.01,overcast clouds
23,23,Majalgaon,IN,19.1500,76.2333,78.26,23,44,7.52,scattered clouds
24,24,Rikitea,PF,-23.1203,-134.9692,78.69,75,8,17.29,clear sky
25,25,Busselton,AU,-33.6500,115.3333,72.00,90,100,3.00,overcast clouds
32,32,Avarua,CK,-21.2078,-159.7750,82.40,83,75,6.91,light rain
33,33,Cabo San Lucas,MX,22.8909,-109.9124,75.20,53,40,11.50,scattered clouds
35,35,Madang,PG,-5.2246,145.7966,81.81,73,92,4.56,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().values.any()

True

In [11]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [16]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Mar Del Plata,AR,71.60,clear sky,-38.0023,-57.5575,
5,Atuona,PF,79.63,light rain,-9.8000,-139.0333,
6,Gewane,ET,74.91,clear sky,10.1664,40.6453,
16,Honiara,SB,81.09,overcast clouds,-9.4333,159.9500,
23,Majalgaon,IN,78.26,scattered clouds,19.1500,76.2333,
24,Rikitea,PF,78.69,clear sky,-23.1203,-134.9692,
25,Busselton,AU,72.00,overcast clouds,-33.6500,115.3333,
32,Avarua,CK,82.40,light rain,-21.2078,-159.7750,
33,Cabo San Lucas,MX,75.20,scattered clouds,22.8909,-109.9124,
35,Madang,PG,81.81,overcast clouds,-5.2246,145.7966,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [25]:
# Check hotel_df for null values
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Mar Del Plata,AR,71.60,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
5,Atuona,PF,79.63,light rain,-9.8000,-139.0333,Villa Enata
16,Honiara,SB,81.09,overcast clouds,-9.4333,159.9500,Heritage Park Hotel
23,Majalgaon,IN,78.26,scattered clouds,19.1500,76.2333,इंद्रायणी ढाबा
24,Rikitea,PF,78.69,clear sky,-23.1203,-134.9692,Pension Maro'i
25,Busselton,AU,72.00,overcast clouds,-33.6500,115.3333,Observatory Guest House
32,Avarua,CK,82.40,light rain,-21.2078,-159.7750,Paradise Inn
33,Cabo San Lucas,MX,75.20,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
35,Madang,PG,81.81,overcast clouds,-5.2246,145.7966,Comfort Inn Hotel
37,Hobart,AU,70.00,few clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [26]:
# Dependecy
import numpy as np
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Mar Del Plata,AR,71.60,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
5,Atuona,PF,79.63,light rain,-9.8000,-139.0333,Villa Enata
16,Honiara,SB,81.09,overcast clouds,-9.4333,159.9500,Heritage Park Hotel
23,Majalgaon,IN,78.26,scattered clouds,19.1500,76.2333,इंद्रायणी ढाबा
24,Rikitea,PF,78.69,clear sky,-23.1203,-134.9692,Pension Maro'i
25,Busselton,AU,72.00,overcast clouds,-33.6500,115.3333,Observatory Guest House
32,Avarua,CK,82.40,light rain,-21.2078,-159.7750,Paradise Inn
33,Cabo San Lucas,MX,75.20,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
35,Madang,PG,81.81,overcast clouds,-5.2246,145.7966,Comfort Inn Hotel
37,Hobart,AU,70.00,few clouds,-42.8794,147.3294,Mantra on Collins Hobart


In [8]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation_Search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'hotel_df' is not defined

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))